# Remove Emty Transcriptions from Trascribed Data Frame

For some reason does some of the emty trascriptions sneek into the trascribed dataframe. This script fixes this.

Libary Imports

In [ ]:
# this scripts task is to go throng all the transcribed files. 
# It checks if there are any missing values in the transcribed files, that is not in the empty transcriptions file.
# It also compar the length to teh assessment files, and assume theses row are all files the model could not transcribe
# They are therefor deemed as empty transcriptions, and given CER = 1.0
import self_made_functions as smf
import pandas as pd
import os

Get the transcribed files, the assessment df, and Initiate the empty list to store the fixed transcriptions.

In [ ]:
# Get the transcribed files to look through
path_transcriptions = './Transcriptions'
lst = os.listdir(path_transcriptions)
lst = [file for file in lst if file.startswith('tran') and file.endswith('.csv')]

# Compar with original assessment
df_assessment, _ = smf.get_correct_df()

# Initialize the empty transcriptions data frame
empty_transcriptions = pd.DataFrame(columns=["file_name",
                                "CER",  # Character Error Rate (CER)
                                "target_word", 
                                "global_score",
                                'model_name', 'version'])

Iterate throng all the transcriptions and find missing values. Remove these from the transcribed data frame, and save them accordantly.

In [59]:
for file in lst:
    df_csv = pd.read_csv(os.path.join(path_transcriptions, file))
    nan_df = df_csv[df_csv.isna('Transcribed').any(axis=1)]
    
    if not nan_df.empty:
        print(f'NaN in {file}')
        df_csv = df_csv.dropna()
        df_csv.to_csv(os.path.join(path_transcriptions, file), index=False)
    else:
        print(f'No NaN')

No NaN
No NaN
No NaN
No NaN
No NaN
No NaN
No NaN
No NaN
No NaN
No NaN
No NaN
No NaN


Now, there is non nan values in the transcribed data frame. Meaning all missing rows in the assessment data frame are empty transcriptions.

In [51]:
# Iterate throng all the transcriptions and find the missing values
for file in lst:
    df_csv = pd.read_csv(os.path.join(path_transcriptions, file))
    model = file.split('_')[1]
    version = file.split('_')[2].split('.')[0]
    
    # Look for rows in the transcription whit NaN values, 
    # and rows in the assessment that is not in the transcription

    missing_rows = df_assessment[~df_assessment['File name'].isin(df_csv['File name'])]
    
    
    
    
    
    # I assume this theses row are all files the model could not transcribe
    # They are deemed as empty transcriptions, and given CER = 1.0
    # Check if the missing values are already in the empty transcriptions file
    
    # Handle the missing rows
    for i, row in missing_rows.iterrows():
        new_row = {"file_name": row['File name'],
                    "CER": 1.0,  # Character Error Rate (CER)
                    "target_word": row['Word'], 
                    "global_score": row['Score'],
                    'model_name': model,
                    'version': version}
        new_row = pd.DataFrame(new_row, index=[0])
        empty_transcriptions = pd.concat([empty_transcriptions, new_row], ignore_index=True)
    
    # handel the NaN / missing values
    for i, row in nan_df.iterrows():
        new_row = {"file_name": row['File name'],
                    "CER": 1.0,  # Character Error Rate (CER)
                    "target_word": row['Word'], 
                    "global_score": row['OG Score'],
                    'model_name': model,
                    'version': version}
        new_row = pd.DataFrame(new_row, index=[0])
        empty_transcriptions = pd.concat([empty_transcriptions, new_row], ignore_index=True)

    bool = missing_rows['File name'].isin(nan_df['File name']).any() # Se if the nan values are in the missing rows
    if bool:
        print('Some of the missing values are in the missing rows\n Make script for this')
        break
    else:
        # remove the nan values from the transcription file
        df_csv_no_nan = df_csv.dropna()
        if len(df_csv) != len(df_csv_no_nan):
            # print(f'NaN gets droped from {file}')
            df_csv_no_nan.to_csv(os.path.join(path_transcriptions, file), index=False)
            df_csv =df_csv_no_nan
        # else: 
            # print(f'No NaN values in {file}')
            
    print(len(df_assessment) == len(df_csv), 'og len:', len(df_assessment), 'csv len:', len(df_csv), 'diff:', len(df_assessment)-len(df_csv))
    print(len(empty_transcriptions), len(empty_transcriptions.drop_duplicates('file_name')))
    
# Save the empty transcriptions
# for version in empty_transcriptions.version.unique():
#     empty_version = empty_transcriptions[empty_transcriptions['version'] == version]
#     empty_version = empty_version.drop(columns=['version'])
#     empty_version.to_csv(os.path.join(path_transcriptions, f'empty_transcription_{version}.csv'), index=False)

/tmp/ipykernel_1518214/2080126791.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  empty_transcriptions = pd.concat([empty_transcriptions, new_row], ignore_index=True)


False og len: 9322 csv len: 9316 diff: 6
6 6
False og len: 9322 csv len: 9218 diff: 104
110 107
False og len: 9322 csv len: 9316 diff: 6
116 109
False og len: 9322 csv len: 9319 diff: 3
119 109
False og len: 9322 csv len: 9316 diff: 6
125 110
False og len: 9322 csv len: 9319 diff: 3
128 110
False og len: 9322 csv len: 9316 diff: 6
134 112
False og len: 9322 csv len: 9282 diff: 40
174 132
False og len: 9322 csv len: 9295 diff: 27
201 146
False og len: 9322 csv len: 9316 diff: 6
207 146
False og len: 9322 csv len: 9218 diff: 104
311 146
False og len: 9322 csv len: 9316 diff: 6
317 146


In [30]:
empty_transcriptions

,file_name,CER,target_word,global_score,model_name
0,a12_graa.wav,1.0,grå,2,nb-whisper-tiny-verbatim
1,d09_svart.wav,1.0,svart,1,nb-whisper-tiny-verbatim
2,d10_svart.wav,1.0,svart,2,nb-whisper-tiny-verbatim
3,a29_lun.wav,1.0,lun,4,nb-whisper-tiny-verbatim
4,a22_mur.wav,1.0,mur,5,nb-whisper-tiny-verbatim
5,a21_gul.wav,1.0,gul,4,nb-whisper-tiny-verbatim


In [ ]:
    empty_transcriptions
    
    
    
    
    print(model, version)
    # print(nan_df['File name'].values)
    
    
    
    # print()
    

    
    

        new_empty_rows = new_empty_rows.drop(['Prosody','Noise/Disruption','Pre-speech noise','Repetition', 'Assessor', 'Pronunciation'], axis=1)
        leaf_row = leaf_row.drop(['Prosody','Noise/Disruption','Pre-speech noise','Repetition', 'Assessor', 'Pronunciation'], axis=1)
        
        # Calulate PER and add CER = 1.0
        new_empty_rows['per'] =  new_empty_rows.apply(lambda x: x['pronScores'].split(' ').count('0')/len(x['pronScores'].split(' ')), axis=1)
        leaf_row['per'] =  leaf_row.apply(lambda x: x['pronScores'].split(' ').count('0')/len(x['pronScores'].split(' ')), axis=1)
        
        new_empty_rows['CER'] = 1.0
        leaf_row['CER'] = 1.0
        
        empty_rows = new_empty_rows.drop(['pronScores'], axis=1)
        leaf_row = leaf_row.drop(['pronScores'], axis=1)
        

In [ ]:

                
            # The following prints show the model has more empty transcription. 
            # After this should df == df_original.
            print(model)
            print(len(df), len(df_original))
            if len(df) != len(df_original):
                print('Bruh')
                missing_rows = df_original[~df_original['File name'].isin(df['File name'])]
                print(len(missing_rows), missing_rows['File name'].values)
                for idx, row in missing_rows.iterrows():
                    new_row = {
                        'File name': row['File name'],
                        'OG word': row['Word'],
                        'idx': idx,
                        'Model': model
                    }
                    rows.append(new_row)
                                
                    # df.to_csv(df_path, index=False)
                    # print(f'Updated original df saved to {df_path}\n')

In [ ]:
        for model in models:
            file_name = df_base + model + '_' + file_version + '.csv'
            df_path = os.path.join(df_dir, file_name)
            df = pd.read_csv(df_path)
            
            # PART 3: Check for missing values in the transcription file
            nan_df = df[df.isna().any(axis=1)]
            if not nan_df.empty:
                print(f'Model: {model}, Version: {file_version}')
                # print('The missing values in the Transcribed file:\n', nan_df, '\n')
                
                # Check if the missing values are already in the empty transcriptions file
                if not nan_df['File name'].isin(mt_df['File name']).any():
                    print('The missing values are not in the empty transcriptions file\n')
                    
                    # PART 4: Add the missing values to the empty transcriptions file
                    rows = []
                    for idx, row in nan_df.iterrows():
                        new_row = {
                            'File name': row['File name'],
                            'OG word': row['Word'],
                            'idx': idx,
                            'Model': model
                        }
                        rows.append(new_row)
                    
                    new_mt_df = pd.DataFrame(rows)
                    # print('Rows to be added to the new dataframe:\n', new_mt_df, '\n')
                    
                    # Update the mt_df and save it
                    finito_mt_df = pd.concat([mt_df, new_mt_df], ignore_index=True)
                    finito_mt_df.to_csv(mt_path, index=False)
                    # print('Updated empty df:\n', finito_mt_df[finito_mt_df['Model'] == model], '\n')
                    
                    # PART 5: Remove the rows with missing values from the original dataframe
                    df = df.dropna()            
            else: 
                print('The missing values are already in the empty transcriptions file\n')
                
                
            # The following prints show the model has more empty transcription. 
            # After this should df == df_original.
            print(model)
            print(len(df), len(df_original))
            if len(df) != len(df_original):
                print('Bruh')
                missing_rows = df_original[~df_original['File name'].isin(df['File name'])]
                print(len(missing_rows), missing_rows['File name'].values)
                for idx, row in missing_rows.iterrows():
                    new_row = {
                        'File name': row['File name'],
                        'OG word': row['Word'],
                        'idx': idx,
                        'Model': model
                    }
                    rows.append(new_row)
                                
                    # df.to_csv(df_path, index=False)
                    # print(f'Updated original df saved to {df_path}\n')


print('Process completed successfully.')

# PART 6: Save the updated empty transcriptions file
# (Already saved within the loop above)


OLD CODE

In [ ]:
import pandas as pd 
import os

# PART 1: Load the data -----------------------------------------------------------------
directory = './Transcriptions'
df_file_name = 'transcriptions_nb-whisper-medium-verbatim_v1.csv'
path = os.path.join(directory, df_file_name)

model_name = df_file_name.split('_')[1]
version = df_file_name.split('_')[2].split('.')[0]

df = pd.read_csv(path)

# PART 2: Check for missing values -------------------------------------------------------
nan_df = df[df.isna().any(axis=1)] # find rows that is nan
print('The missing values in the Trascriped file:\n', nan_df, '\n')

# PART 3: Check if the missing values are in the empty transcriptions file ----------------
# Find if the Emty rows in the dataframe is alreaddy added in the emty transcriptions file
mt_file_name = 'empty_transcription_v2.csv'
mt_path = os.path.join(directory, mt_file_name)

mt_df = pd.read_csv(mt_path)

# Check if the mt df conatins the mt transcriptions 
print('Current rows in the emty dataframe:\n', mt_df[mt_df['Model'] == model_name], '\n')

# If they are not in the file, add them to the file

if nan_df['File name'].values not in mt_df['File name'].values:
    print('The missing values are not in the empty transcriptions file\n')
    

    # PART 4: Add the missing values to the empty transcriptions file --------------------------
    # Find the relevant info in the mt datafram
    file_name = nan_df['File name'].values
    OG_word = nan_df['Word'].values
    idx = nan_df.index.values
    model_name = model_name

    # Add the new information to the mt dataframe
    new_mt_df = {
            'File name': row['File name'],
            'OG word': row['Word'],
            'idx' : idx,
            'Model': model_name
        }

    new_mt_df = pd.DataFrame(new_mt_df)
    print('Rows to the new dataframe:\n', new_mt_df, '\n')

    # Utpdate the mt_df to be correct og save it
    finito_mt_df = pd.concat([mt_df, new_mt_df], ignore_index=True)
    print('finito empty df:\n', finito_mt_df[finito_mt_df['Model'] == model_name], '\n')

# PART 5: remove the rows from the original dataframe and save it --------------------------
df = df.dropna()
# df.to_csv(path, index=False)

# PART 6: Save the updated empty transcriptions file ---------------------------------------
# finito_mt_df.to_csv(mt_path, index=False)

In [ ]:
import os
import pandas as pd

# PART 1: Define directories and base filenames
mt_dir = './Transcriptions/Empty_Transcriptions/Wrong'
df_dir = './Transcriptions'
df_base = 'transcriptions_'

# PART 2: Iterate through empty transcription files
for file in os.listdir(mt_dir):
    if file.startswith('empty') and file.endswith('.csv'):
        file_version = file.split('_')[2].split('.')[0]
        mt_path = os.path.join(mt_dir, file)
        mt_df = pd.read_csv(mt_path)
        
        models = mt_df['Model'].unique()
        
        for model in models:
            file_name = df_base + model + '_' + file_version + '.csv'
            df_path = os.path.join(df_dir, file_name)
            df = pd.read_csv(df_path)
            
            # PART 3: Check for missing values in the transcription file
            nan_df = df[df.isna().any(axis=1)]
            if not nan_df.empty:
                print(f'Model: {model}, Version: {file_version}')
                # print('The missing values in the Transcribed file:\n', nan_df, '\n')
                
                # Check if the missing values are already in the empty transcriptions file
                if not nan_df['File name'].isin(mt_df['File name']).any():
                    print('The missing values are not in the empty transcriptions file\n')
                    
                    # PART 4: Add the missing values to the empty transcriptions file
                    rows = []
                    for idx, row in nan_df.iterrows():
                        new_row = {
                            'File name': row['File name'],
                            'OG word': row['Word'],
                            'idx': idx,
                            'Model': model
                        }
                        rows.append(new_row)
                    
                    new_mt_df = pd.DataFrame(rows)
                    # print('Rows to be added to the new dataframe:\n', new_mt_df, '\n')
                    
                    # Update the mt_df and save it
                    finito_mt_df = pd.concat([mt_df, new_mt_df], ignore_index=True)
                    finito_mt_df.to_csv(mt_path, index=False)
                    # print('Updated empty df:\n', finito_mt_df[finito_mt_df['Model'] == model], '\n')
                    
                    # PART 5: Remove the rows with missing values from the original dataframe
                    df = df.dropna()
                    df.to_csv(df_path, index=False)
                    # print(f'Updated original df saved to {df_path}\n')
            else: 
                print('The missing values are already in the empty transcriptions file\n')

print('Process completed successfully.')

# PART 6: Save the updated empty transcriptions file
# (Already saved within the loop above)
